<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Add-price-&amp;-ongkir" data-toc-modified-id="Add-price-&amp;-ongkir-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Add price &amp; ongkir</a></span></li><li><span><a href="#Filter-products-with-dynamic-pricing" data-toc-modified-id="Filter-products-with-dynamic-pricing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Filter products with dynamic pricing</a></span></li><li><span><a href="#Aggregate-sales-and-price-per-week" data-toc-modified-id="Aggregate-sales-and-price-per-week-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Aggregate sales and price per week</a></span></li><li><span><a href="#filter-with-minimum-sales-timestamp" data-toc-modified-id="filter-with-minimum-sales-timestamp-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>filter with minimum sales timestamp</a></span></li><li><span><a href="#Interpolate-periodical-features" data-toc-modified-id="Interpolate-periodical-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interpolate periodical features</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm

import datetime

In [2]:
items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')
name_translation_df = pd.read_csv('../data/product_category_name_translation.csv')

### Add price & ongkir

In [3]:
data = orders_df[['order_id', 'order_purchase_timestamp', 'order_status']]
data

,order_id,order_purchase_timestamp,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,delivered
99438,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,delivered
99439,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,delivered


In [4]:
data = data.merge(items_df[['order_id', 'product_id', 'price', 'freight_value']], on='order_id')
data

,order_id,order_purchase_timestamp,order_status,product_id,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,delivered,87285b34884572647811a353c7ac498a,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,delivered,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,delivered,aa4383b373c6aca5d8797843e5594415,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,delivered,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,delivered,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72
...,...,...,...,...,...,...
112645,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,delivered,f1d4ce8c6dd66c47bbaa8c6781c2a923,174.90,20.10
112646,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,delivered,b80910977a37536adeddd63663f916ad,205.99,65.02
112647,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,delivered,d1c427060a0f73f6b889a5c7c61f2ac4,179.99,40.59
112648,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,delivered,d1c427060a0f73f6b889a5c7c61f2ac4,179.99,40.59


### Filter products with dynamic pricing

In [7]:
items_price_std = data.groupby('product_id').price.std()
product_ids = items_price_std[items_price_std.notna() & items_price_std!=0].index
print('total product id:', data.product_id.nunique())
print('total product id with dynamic pricing:', product_ids.shape[0])

data = data.set_index('product_id').loc[product_ids]
data

total product id: 32951
total product id with dynamic pricing: 5900


,order_id,order_purchase_timestamp,order_status,price,freight_value
product_id,,,,,
00210e41887c2a8ef9f791ebc780cc36,1fd6c29ecb9dd8b65359f54c969105d3,2017-05-22 12:15:02,delivered,32.98,8.27
00210e41887c2a8ef9f791ebc780cc36,eb24356203f63304c71d81f2c841a02a,2017-06-04 12:44:27,delivered,33.99,8.27
00210e41887c2a8ef9f791ebc780cc36,eb24356203f63304c71d81f2c841a02a,2017-06-04 12:44:27,delivered,33.99,8.27
00210e41887c2a8ef9f791ebc780cc36,226975521c585d7bb98996f83ccdde59,2017-06-26 16:15:10,delivered,33.99,15.10
00210e41887c2a8ef9f791ebc780cc36,2d8e71bf7d31a41d40f122c8173fdf06,2017-06-01 16:05:36,delivered,32.98,15.10
...,...,...,...,...,...
fffdb2d0ec8d6a61f0a0a0db3f25b441,d6c1ed12e8eeaa15e9d5c83c79598f85,2018-08-10 17:15:07,delivered,34.99,7.51
fffdb2d0ec8d6a61f0a0a0db3f25b441,beaf95465b542dea76df78b2fb86a97a,2018-04-19 10:12:12,delivered,34.99,18.23
fffdb2d0ec8d6a61f0a0a0db3f25b441,dd38c566a34ff9deb93ff05d7bd6d0aa,2018-05-01 17:31:41,delivered,34.99,7.39


### Aggregate sales and price per week

In [8]:
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp']) - pd.to_timedelta(7, unit='d')

In [9]:
data['sales'] = 1
data_sales = data.groupby(['product_id', pd.Grouper(key='order_purchase_timestamp', freq='W-MON')])['sales'] \
       .sum() \
       .reset_index() \
       .sort_values('order_purchase_timestamp')
data_sales

,product_id,order_purchase_timestamp,sales
26736,c1488892604e4ba5cff5b4eb4d595400,2016-08-29,1
12741,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-12,3
25417,b72b39418216e944bb34e35f4d3ea8c7,2016-09-26,1
21861,9c7bdf67b06b419aefb93cfdfc96c55d,2016-09-26,1
11475,52b668edc0d0c20cd9319bfda4019597,2016-10-03,1
...,...,...,...
31079,e1e0040c906cc890d032092141a9b6de,2018-08-27,1
7540,37f4d0bf85fbf875c920d460766d6a5c,2018-08-27,1
28983,d04857e7b4b708ee8b8b9921163edba3,2018-08-27,1
25380,b6b76b074ed0d77d0f3443b12d8adb5e,2018-08-27,1


In [10]:
data_price = data.groupby(['product_id', pd.Grouper(key='order_purchase_timestamp', freq='W-MON')])['price'] \
       .mean() \
       .reset_index() \
       .sort_values('order_purchase_timestamp')
data_price

,product_id,order_purchase_timestamp,price
26736,c1488892604e4ba5cff5b4eb4d595400,2016-08-29,39.99
12741,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-12,44.99
25417,b72b39418216e944bb34e35f4d3ea8c7,2016-09-26,36.49
21861,9c7bdf67b06b419aefb93cfdfc96c55d,2016-09-26,21.50
11475,52b668edc0d0c20cd9319bfda4019597,2016-10-03,59.90
...,...,...,...
31079,e1e0040c906cc890d032092141a9b6de,2018-08-27,79.49
7540,37f4d0bf85fbf875c920d460766d6a5c,2018-08-27,30.00
28983,d04857e7b4b708ee8b8b9921163edba3,2018-08-27,84.99
25380,b6b76b074ed0d77d0f3443b12d8adb5e,2018-08-27,99.00


In [11]:
data = data_sales.merge(data_price, on=['product_id', 'order_purchase_timestamp'])
data

,product_id,order_purchase_timestamp,sales,price
0,c1488892604e4ba5cff5b4eb4d595400,2016-08-29,1,39.99
1,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-12,3,44.99
2,b72b39418216e944bb34e35f4d3ea8c7,2016-09-26,1,36.49
3,9c7bdf67b06b419aefb93cfdfc96c55d,2016-09-26,1,21.50
4,52b668edc0d0c20cd9319bfda4019597,2016-10-03,1,59.90
...,...,...,...,...
35373,e1e0040c906cc890d032092141a9b6de,2018-08-27,1,79.49
35374,37f4d0bf85fbf875c920d460766d6a5c,2018-08-27,1,30.00
35375,d04857e7b4b708ee8b8b9921163edba3,2018-08-27,1,84.99
35376,b6b76b074ed0d77d0f3443b12d8adb5e,2018-08-27,1,99.00


### filter with minimum sales timestamp

In [14]:

n_price_point = data.groupby('product_id').order_purchase_timestamp.count()
product_ids = n_price_point[n_price_point > 8].index
data = data.set_index('product_id').loc[product_ids].reset_index()

print('total product id left:', product_ids.shape[0])
print('rotal row left:', data.shape[0])

total product id left: 1076
rotal row left: 17534


In [20]:
data

,product_id,order_purchase_timestamp,sales,price
0,005030ef108f58b46b78116f754d8d38,2017-10-16,1,13.99
1,005030ef108f58b46b78116f754d8d38,2017-11-13,3,13.99
2,005030ef108f58b46b78116f754d8d38,2017-11-20,1,13.99
3,005030ef108f58b46b78116f754d8d38,2018-01-15,1,13.99
4,005030ef108f58b46b78116f754d8d38,2018-01-29,1,13.99
...,...,...,...,...
17529,ffc9caf33e2d1e9f44e3e06da19085f7,2018-05-28,1,199.89
17530,ffc9caf33e2d1e9f44e3e06da19085f7,2018-06-04,1,224.00
17531,ffc9caf33e2d1e9f44e3e06da19085f7,2018-06-11,1,224.00
17532,ffc9caf33e2d1e9f44e3e06da19085f7,2018-06-18,1,224.00


### Interpolate periodical features

In [79]:
def get_interpolated_timestamp(timestamp):
    """ Fill in missing mondays between timestamp gap
    
    NOTE: the timestamp series must already be sorted!
    """
    
    return pd.date_range(start=timestamp.iloc[0], end=timestamp.iloc[-1], freq='W-MON').to_frame(name='order_purchase_timestamp')

def apply_interpolation(product_sales_history, product_id):
    """ Fill in the gaps between sales & price timestamps """
    
    product_sales_history = product_sales_history.sort_values('order_purchase_timestamp')
    timestamp = get_interpolated_timestamp(product_sales_history['order_purchase_timestamp'])
    product_sales_history = product_sales_history.merge(timestamp, 
                                                        on='order_purchase_timestamp', 
                                                        how='outer').sort_values('order_purchase_timestamp')
    product_sales_history['sales'] = product_sales_history['sales'].fillna(0)
    product_sales_history['price'] = product_sales_history['price'].fillna(method='ffill')    
    product_sales_history['product_id'] = product_id
    
    return product_sales_history

In [80]:
from tqdm import tqdm

interpolated_data = []
for product_id, product in tqdm(data.groupby('product_id')):
    interpolated_data.append(apply_interpolation(product, product_id))
    

100%|██████████████████████████████████████████████| 1076/1076 [00:07<00:00, 143.34it/s]


In [81]:
data = pd.concat(interpolated_data, axis=0)
data

,product_id,order_purchase_timestamp,sales,price
0,005030ef108f58b46b78116f754d8d38,2017-10-16,1.0,13.99
10,005030ef108f58b46b78116f754d8d38,2017-10-23,0.0,13.99
11,005030ef108f58b46b78116f754d8d38,2017-10-30,0.0,13.99
12,005030ef108f58b46b78116f754d8d38,2017-11-06,0.0,13.99
1,005030ef108f58b46b78116f754d8d38,2017-11-13,3.0,13.99
...,...,...,...,...
26,ffc9caf33e2d1e9f44e3e06da19085f7,2018-07-02,0.0,224.00
27,ffc9caf33e2d1e9f44e3e06da19085f7,2018-07-09,0.0,224.00
28,ffc9caf33e2d1e9f44e3e06da19085f7,2018-07-16,0.0,224.00
29,ffc9caf33e2d1e9f44e3e06da19085f7,2018-07-23,0.0,224.00


In [84]:
print('total product id left:', product_ids.shape[0])
print('rotal row left:', data.shape[0])

total product id left: 1076
rotal row left: 50278


In [87]:
data.groupby('product_id').order_purchase_timestamp.count().mean()

46.726765799256505

In [83]:
data.to_csv('../data/prepared/interpolated_demand_history.csv')